In [1]:
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import psycopg2
import atexit

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Conexion con la base de datos 

In [2]:
def close_database():
  if connection:
    cursor.close()
    connection.close()
    print("Database connection is closed")

atexit.register(close_database)

try:
  connection = psycopg2.connect(
    user = "fwlnuygvpzvpis",
    password = "da023af44eb4d87c3587ddd2edb09d2a43263dbb4405a86ec99dc34d05baaad8",
    host = "ec2-54-197-48-79.compute-1.amazonaws.com",
    port = "5432",
    database = "d54hmasrunnmlg"
  )
  cursor = connection.cursor()
  cursor.execute("SELECT version();")
  record = cursor.fetchone()
  print("You are connected to - ", record, "\n")

except (Exception, psycopg2.Error) as error:
  print ("Error while connecting to PostgreSQL", error)
  exit()

You are connected to -  ('PostgreSQL 12.2 (Ubuntu 12.2-2.pgdg16.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 5.4.0-6ubuntu1~16.04.12) 5.4.0 20160609, 64-bit',) 



In [3]:
tabla_impresoras = pd.read_sql_query("select id as id_impresora,tipo_color,wifi,escaner, precio from impresoras ",con=connection)
print(tabla_impresoras)

     id_impresora     tipo_color wifi escaner   precio
0           10662          Color  Yes      No   129.99
1           10663          Color  Yes      No   159.99
2           10664          Color  Yes      No   199.99
3           10665          Color  Yes      No    69.99
4           10666          Color  Yes      No    99.99
..            ...            ...  ...     ...      ...
130         10767     Monochrome   No      No   648.69
131         10821     Monochrome   No      No  5399.00
132         10822     Monochrome   No      No  5399.00
133         10736  Monochromatic  Yes      No   348.99
134         10738  Monochromatic  Yes      No   198.99

[135 rows x 5 columns]


In [4]:
tabla_impresoras_calificadas = pd.read_sql_query('select user_id as id_user, impresora_id as id_impresora, calificacion as puntuacion from calificaciones',con=connection)
print(tabla_impresoras_calificadas)

    id_user  id_impresora  puntuacion
0        10         10663         3.0
1        10         10817         3.0
2        12         10662         5.0
3        14         10662         5.0
4        14         10663         5.0
..      ...           ...         ...
57       19         10694         3.0
58       19         10696         3.0
59       19         10699         3.0
60       19         10697         2.0
61       19         10703         4.0

[62 rows x 3 columns]


# Sitema de Recomendacion Basado en Contenido 
# Sistema de Recomendacion Colaborativo 
# Sistema Hibrido

In [27]:
# la siguiente funcion devuelve un dataframe con una columna 
# por tipo_colo, columna con valores binarios de wifi y escaner
def preparando_matriz(tabla_impresoras):
  df = pd.get_dummies(tabla_impresoras, columns = ["tipo_color"])
  df = pd.get_dummies(df, columns = ["wifi"])
  columnas1 = df.columns.values
  if "wifi_Yes" not in columnas1:
    df["wifi_Yes"] = 0
  if "wifi_No" in columnas1:
    df = df.drop(["wifi_No"],axis=1)
  df = pd.get_dummies(df, columns = ["escaner"])
  columnas2 = df.columns.values
  if "escaner_Yes" not in columnas2:
    df["escaner_Yes"] = 0
  if "escaner_No" in columnas2:
    df = df.drop(["escaner_No"],axis=1)

  return df

#función que devuelve la columan de calificaciones del usuario
#y la matriz de impresoras que fueron calificadas por el usuario
def matriz_y_puntuacion_user(tabla_impresoras_calificadas,tabla_impresoras, id_usuario):
    df = tabla_impresoras_calificadas[tabla_impresoras_calificadas['id_user'] == id_usuario]
    puntuaciones = df.loc[:, ["id_impresora","puntuacion"]]
    impresoras_calificadas = df.loc[:, "id_impresora"]
    tabla_impresoras_ratings = tabla_impresoras[tabla_impresoras["id_impresora"].isin(impresoras_calificadas)]

    return tabla_impresoras_ratings, puntuaciones 

def calcular_perfil_usuario(matriz, ratings_usuario):
  for i in range(1, len(matriz.columns)):
    columna = matriz.columns[i]
    #multiplicando matriz por puntuaciones
    matriz[columna]= matriz[columna].multiply(ratings_usuario.puntuacion.values)
  
  perfil = matriz.sum()

  #perfil_usuario_normalizado
  suma_puntuacion = ratings_usuario.puntuacion.sum()
  suma_perfil= perfil.iloc[1:].sum()

  wifi = perfil["wifi_Yes"]/suma_puntuacion
  escaner = perfil["escaner_Yes"]/suma_puntuacion
  precio = perfil["precio"]/suma_puntuacion

  tiposColor = perfil.drop(["precio","escaner_Yes", "wifi_Yes", "id_impresora"])
  suma_tipoColor = tiposColor.sum()
  tiposColorNormalizado = tiposColor.iloc[0:]/suma_tipoColor
  columnasTipo = tiposColor.index

  dataPrecio = pd.DataFrame({"precio": [precio]})
  
  data = pd.DataFrame(columns=columnasTipo)
  data.loc[len(data)]=tiposColorNormalizado 
  data["wifi_Yes"] = wifi
  data["escaner_Yes"] = escaner

  perfil_user_normalizado = pd.concat([dataPrecio, data], axis=1)

  return matriz, perfil, perfil_user_normalizado   


def impresoras_no_calificadas(tabla_impresoras,tabla_impresoras_calificadas, id_usuario):
  df = tabla_impresoras_calificadas[tabla_impresoras_calificadas['id_user'] == id_usuario]
  impresoras_calificadas = df.loc[:, "id_impresora"]
  tabla_impresoras_not_ratings = tabla_impresoras[~tabla_impresoras.id_impresora.isin(impresoras_calificadas)]

  return tabla_impresoras_not_ratings

def minimos_y_maximos_precio(tabla_impresoras_not_ratings):
  maximo = np.nanmax(tabla_impresoras_not_ratings.loc[:, "precio"].values)
  minimo = np.nanmin(tabla_impresoras_not_ratings.loc[:, "precio"].values)

  return maximo, minimo


def normalizar_matriz_not_ratings(tabla_impresoras_not_ratings, maximo, minimo):
  tabla_impresoras_not_ratings['precio'] = tabla_impresoras_not_ratings.apply(lambda row : normalize(row['precio'],maximo, minimo), axis = 1) 
  
  return tabla_impresoras_not_ratings

def normalize(x, maximo, minimo): 
    x_new = ((x - minimo)*(1-0) / (maximo - minimo)) + 0   
    return x_new   

def agregar_columnas_perfilUserNormalizado(matrizNormalizada, perfilUsuarioNormalizado):
  if (len(matrizNormalizada.columns) == len(perfilUsuarioNormalizado.columns) + 1):
    perfilUserNew = perfilUsuarioNormalizado.iloc[0].values
    return perfilUserNew
  else:
    columnasMatriz = matrizNormalizada.columns.values
    columnasPerfil = perfilUsuarioNormalizado.columns.values
    perfilUserNew = []
    for i in range(1, len(columnasMatriz)):
      if (columnasMatriz[i] in columnasPerfil):
        perfilUserNew.append(perfilUsuarioNormalizado[columnasMatriz[i]].values[0])
      else: 
        perfilUserNew.append(0)
    
    return perfilUserNew

def distaciaEuclidiana(x,y): 
  return np.sqrt(np.sum((x-y)**2))

def encontrar_distancia_euclidiana(tablaImpresoraNoCalificadas, perfilUserNormalizado):
  listaDistanciaEuclidiana = []
  for indice, fila in tablaImpresoraNoCalificadas.iterrows():
    filaTablaImpresoras = fila.values
    filaTablaImpresoras = np.delete(filaTablaImpresoras, 0)
    dist_a_b = distaciaEuclidiana(perfilUserNormalizado,filaTablaImpresoras)
    listaDistanciaEuclidiana.append(dist_a_b)

  tablaImpresoraNoCalificadas["distanciaEuclidiana"] = listaDistanciaEuclidiana
  return tablaImpresoraNoCalificadas

def mainContenido(tabla_impresoras_calificadas, tabla_impresoras,id_usuario):
  #condicional para saber si el usuario tiene calificaciones
  tabla_impresoras2, ratings_usuario = matriz_y_puntuacion_user(tabla_impresoras_calificadas, tabla_impresoras,id_usuario)
  ratings_usuario = ratings_usuario.sort_values('id_impresora',ascending=True)
  tabla_impresoras2 = tabla_impresoras2.sort_values('id_impresora',ascending=True)
  matriz = preparando_matriz(tabla_impresoras2)
 
  #perfil_usuario normalizado
  matriz_x_calificaciones, perfil, perfil_normalizado = calcular_perfil_usuario(matriz, ratings_usuario)
  
  columnas = perfil_normalizado.columns.values
  valores = perfil_normalizado.iloc[0].to_list()
  
  matriz_impresoras_not_ratings = impresoras_no_calificadas(tabla_impresoras,tabla_impresoras_calificadas,id_usuario)
  matriz_impresoras_not_ratings= preparando_matriz(matriz_impresoras_not_ratings)
  maximo, minimo = minimos_y_maximos_precio(matriz_impresoras_not_ratings)
  matriz_normalizada = normalizar_matriz_not_ratings(matriz_impresoras_not_ratings, maximo, minimo)
  perfil_usuarioNormalizado = normalizar_matriz_not_ratings(perfil_normalizado, maximo, minimo)
  listaPerfil_usuarioNormalizado = agregar_columnas_perfilUserNormalizado(matriz_normalizada,perfil_usuarioNormalizado)  
  dataframeFinal = encontrar_distancia_euclidiana(matriz_normalizada,listaPerfil_usuarioNormalizado)
  #print("Recomendacion Ordenado de menor distancia a mayor ")
  dataframeFinal = dataframeFinal.sort_values('distanciaEuclidiana',ascending=True) 

  return columnas, valores, dataframeFinal

#-----------------------------------------------------------------------------
# aca termina la recomendacion basada en contenido
#-----------------------------------------------------------------------------


Mean = tabla_impresoras_calificadas.groupby(by="id_user",as_index=False)['puntuacion'].mean()
Rating_avg = pd.merge(tabla_impresoras_calificadas,Mean,on='id_user')
Rating_avg['adg_rating']=Rating_avg['puntuacion_x']-Rating_avg['puntuacion_y']
check = pd.pivot_table(Rating_avg,values='puntuacion_x',index='id_user',columns='id_impresora')
final = pd.pivot_table(Rating_avg,values='adg_rating',index='id_user',columns='id_impresora')
final_impresoras = final.fillna(final.mean(axis=0))

# Reemplazo de NaN por usuario Promedio
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

# similitud de usuario al reemplazar NAN por avg de usuario
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index

cosine = cosine_similarity(final_impresoras)
np.fill_diagonal(cosine, 0 )
similarity_with_impresoras = pd.DataFrame(cosine,index=final_impresoras.index)
similarity_with_impresoras.columns=final_user.index
similarity_with_impresoras.head() 

def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

# Los 3 mejores vecinos para cada usuario
sim_user_30_u = find_n_neighbours(similarity_with_user,3)

# Los 3 mejores vecinos para cada usuario
sim_user_30_m = find_n_neighbours(similarity_with_impresoras,3)

Rating_avg = Rating_avg.astype({"id_impresora": str})
Impresora_user = Rating_avg.groupby(by = 'id_user')['id_impresora'].apply(lambda x:','.join(x))

def User_item_score1(user):
    Impresora_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Impresora_user[Impresora_user.index.isin(b)]
    l = ','.join(d.values)
    Impresora_seen_by_similar_users = l.split(',')
    Impresora_under_consideration = list(set(Impresora_seen_by_similar_users)-set(list(map(str, Impresora_seen_by_user))))
    Impresora_under_consideration = list(map(int, Impresora_under_consideration)) # donde esta str es int
    score = []
    for item in Impresora_under_consideration:
        c = final_impresoras.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['id_user'] == user,'puntuacion'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_impresoras.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'id_impresora':Impresora_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(10)
    Impresoras_Name = top_5_recommendation.merge(tabla_impresoras, how='inner', on='id_impresora')
    Impresoras_Names = Impresoras_Name.id_impresora.values.tolist()
    return Impresoras_Names
    
#----------------------------------------------------------------------------
# aca termina la recomendacion Colaborativa User-user
#---------------------------------------------------------------------------------

def mainHibrido(impresoras_calificadas,recomendacion_basado_contenido,recomendacion_colaborativo):
  Hibrido = pd.DataFrame()
  impresoras_calificadas = tabla_impresoras_calificadas.id_impresora.unique()

  Hibrido["id_impresoras"] = impresoras_calificadas
  Hibrido = Hibrido.sort_values('id_impresoras',ascending=True)

  df2=tabla_impresoras_calificadas.groupby("id_impresora").count()[["puntuacion"]]
  Hibrido["puntuacion_count"] = df2["puntuacion"].values

  df3=tabla_impresoras_calificadas.groupby("id_impresora").mean()[["puntuacion"]]
  Hibrido["puntuacion_average"] = df3["puntuacion"].values

  v = Hibrido["puntuacion_count"]
  R = Hibrido["puntuacion_average"]
  C = Hibrido["puntuacion_average"].mean()
  m = Hibrido["puntuacion_count"].quantile(0.70)

  Hibrido['weighted_average']=((R*v)+ (C*m))/(v+m)

  BasadoEnContenido  = recomendacion_basado_contenido.to_list()

  Colaborativo = recomendacion_colaborativo

  for id in Colaborativo:
    if id not in BasadoEnContenido:
      BasadoEnContenido.append(id)

  Hibrido = Hibrido[Hibrido.id_impresoras.isin(BasadoEnContenido)]
  Hibrido = Hibrido.sort_values('weighted_average',ascending=False)
  recomendacionFinal = Hibrido["id_impresoras"]
  recomendacionFinal = recomendacionFinal.to_list()
  for id in BasadoEnContenido:
    if id not in recomendacionFinal:
      recomendacionFinal.append(id)

  return recomendacionFinal
#----------------------------------------------------------------------------
# aca termina la recomendacion Hibrida
#---------------------------------------------------------------------------------





if __name__ == "__main__":
  # variable que recibe el id del usuario que ingresa a la app
  id_usuario = 14

  columnaPerfilUser, valorPerfilUser, dataframeFinal = mainContenido(tabla_impresoras_calificadas, tabla_impresoras,id_usuario)
  recomendacion_basado_contenido = dataframeFinal["id_impresora"].head(10)
  recomendacion_colaborativo = User_item_score1(id_usuario)
  recomendacionFinal = mainHibrido(impresoras_calificadas,recomendacion_basado_contenido,recomendacion_colaborativo)

  print("perfil de usuario ")
  print(columnaPerfilUser)
  print(valorPerfilUser)
  print()
  print("lista de recomendacion Final (id_impresoras) ")
  print("SOLO MOSTRAR 10 ")
  print(recomendacionFinal)
  

perfil de usuario 
['precio' 'tipo_color_Color' 'tipo_color_Monochromatic'
 'tipo_color_Monochrome' 'wifi_Yes' 'escaner_Yes']
[288.3308928571428, 0.7678571428571429, 0.07142857142857142, 0.16071428571428573, 0.8392857142857143, 0.08928571428571429]

lista de recomendacion Final (id_impresoras) 
SOLO MOSTRAR 10 
[10809, 10703, 10745, 10801, 10697, 10694, 10696, 10817, 10699, 10687, 10716, 10701, 10726, 10711, 10693, 10732]
